In [8]:

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.3/107.7 GB disk)


In [9]:

!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CPU

100% 14.1M/14.1M [00:00<00:00, 118MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/2 /content/yolov5/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 494.0ms
image 2/2 /content/yolov5/yolov5/data/images/zidane.jpg: 384x640 2 persons, 2 ties, 472.2ms
Speed: 2.1ms pre-process, 483.1ms inference, 4.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [10]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:05<00:00, 150MB/s]


In [11]:

# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half


val: data=/content/yolov5/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning /content/yolov5/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:03<00:00, 1325.94it/s]
val: New cache created: /content/yolov5/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/157 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/yolov5/yolov5/val.py", line 438, in <module>
    main(opt)
  File "/content/yolov5/yolov5/val.py", line 409, in

In [12]:

# # YOLOv5 PyTorch HUB Inference (DetectionModels only)
# import torch

# # Load YOLOv5 model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)

# # Path to the image
# im = '/content/tr3.jpeg'

# # Perform inference
# results = model(im)

# # Display the results
# results.show()

# # Save the results
# results.save()

# # Count the number of objects detected
# object_count = len(results.pandas().xyxy[0])

# print("Number of objects detected:", object_count)


In [13]:
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)

# Path to the image
im = '/content/tr3.jpeg'

# Perform inference
results = model(im)

# Define the object classes you want to count
object_classes = ['refrigerator', 'chair', 'bed', 'dining table', 'tv', 'bottle', 'sink', 'remote', 'bowl', 'cup', 'spoon', 'fork','wine glass','glass','broccoli']

# Define probability threshold
prob_threshold = 0.2

# Count the number of each type of object detected
object_counts = {obj_class: 0 for obj_class in object_classes}

for detection in results.pred[0]:
    prob = detection[4].item()  # Probability of detection
    if prob >= prob_threshold:
        obj_class = results.names[int(detection[-1])]
        if obj_class in object_classes:
            object_counts[obj_class] += 1

# Display the count for each type of object
for obj_class, count in object_counts.items():
    print(f"Number of {obj_class}: {count}")



Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Number of refrigerator: 0
Number of chair: 0
Number of bed: 0
Number of dining table: 0
Number of tv: 0
Number of bottle: 0
Number of sink: 0
Number of remote: 0
Number of bowl: 0
Number of cup: 0
Number of spoon: 0
Number of fork: 0
Number of wine glass: 2
Number of glass: 0
Number of broccoli: 3


In [14]:
import pickle

# Assuming 'model' is your trained YOLOv5 model

# Define the path where you want to save the model
model_path = '/content/ObjectDetection.pkl'

# Save the model
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved to '{model_path}'")



Model saved to '/content/ObjectDetection.pkl'
